In [6]:
import matplotlib
matplotlib.use('nbagg')
import matplotlib.animation as anm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import math
import numpy as np

In [7]:
class World:
    def __init__(self,debug=False):
        self.objects = []
        self.debug = debug
        
    def append(self,obj):
        self.objects.append(obj)
        
    def one_step(self,frame,elems,ax):
        #elems.clear()
        print(elems)
        while elems: elems.pop().remove()
        elems.append(ax.text(-4.4,4.5, "t= "+str(frame),fontsize=10))
        for obj in self.objects:
            obj.draw(ax,elems)
            if hasattr(obj, "one_step"): obj.one_step(1.0)
    
    def draw(self):
        fig = plt.figure(figsize=(4,4))
        ax=fig.add_subplot(111)
        ax.set_aspect('equal')
        ax.set_xlim(-5,5)
        ax.set_ylim(-5,5)
        ax.set_xlabel("X",fontsize=10)
        ax.set_ylabel("Y",fontsize=10)
        
        elems = []
        
        if self.debug:
            for frame in range(3): self.one_step(frame,elems,ax)
        else:
            self.ani = anm.FuncAnimation(fig=fig,func=self.one_step,frames=10,fargs=(elems,ax),interval=1000,repeat=False)
            plt.show()

In [8]:
class IdealRobot:
    def __init__(self, pose, agent=None, color="Black"):
        self.pose = pose
        self.r = 0.2
        self.color = color
        self.agent = agent
        self.poses = [pose]

    def draw(self,ax,elems):
        x,y,theta = self.pose
        xn=x+self.r*math.cos(theta)
        yn=y+self.r*math.sin(theta)
        elems+=ax.plot([x,xn],[y,yn],color=self.color)
        c = patches.Circle(xy=(x,y),radius=self.r,fill=False,color=self.color)
        elems.append(ax.add_patch(c))
        
        self.poses.append(self.pose)
        elems += ax.plot([e[0] for e in self.poses], [e[1] for e in self.poses], linewidth=0.5, color="black")
        
    @classmethod
    def state_transition(cls, nu, omega, time, pose):
        t0 = pose[2]
        if math.fabs(omega) < 1e-10:
            return pose + np.array([nu*math.cos(t0),
                                    nu*math.sin(t0),
                                    omega]) * time
        else:
            return pose + np.array([nu/omega*( math.sin(t0 + omega*time) - math.sin(t0)),
                                     nu/omega*(-math.cos(t0 + omega*time) + math.cos(t0)),
                                   omega*time])

    def one_step(self,time_interval):
        if not self.agent: return
        nu, omega = self.agent.decision()
        self.pose = self.state_transition(nu, omega, time_interval, self.pose)

In [9]:
class Agent:
    def __init__(self, nu, omega):
        self.nu =nu
        self.omega = omega
        
    def decision(self, observation=None):
        return self.nu, self.omega

In [10]:
world=World(debug=True)

straight = Agent(0.2,0.0)
circling = Agent(0.2, 10.0/180*math.pi)
robot1=IdealRobot(np.array([-1,1,math.pi/3]).T,straight)
robot2=IdealRobot(np.array([-2,-1,math.pi/5*6]).T,circling)
world.append(robot1)
world.append(robot2)
world.draw()

[]
[Text(-4.4, 4.5, 't= 0'), <matplotlib.lines.Line2D object at 0x01495340>, <matplotlib.patches.Circle object at 0x01495280>, <matplotlib.lines.Line2D object at 0x014955F8>, <matplotlib.lines.Line2D object at 0x01495760>, <matplotlib.patches.Circle object at 0x11297DC0>, <matplotlib.lines.Line2D object at 0x014959E8>]
[Text(-4.4, 4.5, 't= 1'), <matplotlib.lines.Line2D object at 0x01495328>, <matplotlib.patches.Circle object at 0x11297A30>, <matplotlib.lines.Line2D object at 0x014955E0>, <matplotlib.lines.Line2D object at 0x014956E8>, <matplotlib.patches.Circle object at 0x065A7CB8>, <matplotlib.lines.Line2D object at 0x01495940>]
